<a href="https://colab.research.google.com/github/MoonlightO2/Fake-Job-Prediction-System/blob/main/Fake_jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

In [17]:
file_path1 = '/content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/All datasets for Data Analysis/All datasets/Fake jobs I.csv'
file_path2 = '/content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/All datasets for Data Analysis/All datasets/Fake jobs II.csv'
pickle_model = '/content/drive/MyDrive/2. Projects/Colab Notebooks/Portfolio/All datasets for Data Analysis/All datasets/fake_job_model.pkl'

In [4]:
data = pd.read_csv(file_path1)
print('Shape: ', data.shape, '\n')
data.columns = data.columns.str.replace('_', ' ').str.title()
print('Columns: ', data.columns.tolist(), '\n')
data.head()

Shape:  (17880, 18) 

Columns:  ['Job Id', 'Title', 'Location', 'Department', 'Salary Range', 'Company Profile', 'Description', 'Requirements', 'Benefits', 'Telecommuting', 'Has Company Logo', 'Has Questions', 'Employment Type', 'Required Experience', 'Required Education', 'Industry', 'Function', 'Fraudulent'] 



,Job Id,Title,Location,Department,Salary Range,Company Profile,Description,Requirements,Benefits,Telecommuting,Has Company Logo,Has Questions,Employment Type,Required Experience,Required Education,Industry,Function,Fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
# Get unique values in each column
unique_values = data.nunique()

# Display unique values and counts
for column_name, count in unique_values.items():
    unique_vals = data[column_name].unique()
    print(f"Column '{column_name}'\n{count} unique values\n", unique_vals, '\n')

Column 'Job Id'
17880 unique values
 [    1     2     3 ... 17878 17879 17880] 

Column 'Title'
11231 unique values
 ['Marketing Intern' 'Customer Service - Cloud Video Production'
 'Commissioning Machinery Assistant (CMA)' ...
 'Senior Financial Analyst (Retail) ' 'Account Director - Distribution '
 'Project Cost Control Staff Engineer - Cost Control Exp - TX'] 

Column 'Location'
3105 unique values
 ['US, NY, New York' 'NZ, , Auckland' 'US, IA, Wever' ...
 'US, CA, los Angeles' 'CA, , Ottawa' 'GB, WSX, Chichester'] 

Column 'Department'
1337 unique values
 ['Marketing' 'Success' nan ... 'Admin - Clerical' 'Administrative Dept'
 'Hospitality'] 

Column 'Salary Range'
874 unique values
 [nan '20000-28000' '100000-120000' '120000-150000' '50000-65000'
 '40000-50000' '60-80' '65000-70000' '75-115' '75000-110000' '17000-20000'
 '16000-28000' '95000-115000' '15000-18000' '50000-70000' '45000-60000'
 '30000-40000' '70000-90000' '10000-14000' '50-110' '28000-45000'
 '0-34300' '35000-40000' '

In [6]:
# Select relevant features
features = ['Title', 'Location', 'Department', 'Salary Range', 'Company Profile', 'Description', 'Requirements', 'Benefits', 'Telecommuting', 'Has Company Logo', 'Has Questions', 'Employment Type', 'Required Experience', 'Required Education', 'Industry', 'Function']

# Drop rows with missing values in the selected features
data = data.dropna(subset=features + ['Fraudulent'])

In [7]:
# Separate features and target variable
X = data[features]
y = data['Fraudulent']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Build a simple pipeline with a text vectorizer and a Decision Tree classifier
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Train the model
pipeline.fit(X_train['Description'], y_train)

# Make predictions on the test set
predictions = pipeline.predict(X_test['Description'])

In [9]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)*100
print(f'Accuracy: {accuracy:.2f}%\n')

# Display classification report
print(classification_report(y_test, predictions))

conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 95.48%

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       142
           1       0.80      0.62      0.70        13

    accuracy                           0.95       155
   macro avg       0.88      0.80      0.84       155
weighted avg       0.95      0.95      0.95       155

Confusion Matrix:
[[140   2]
 [  5   8]]


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import pickle

In [11]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Train the model
pipeline.fit(X_train['Description'], y_train)

with open(pickle_model, 'wb') as model_file:
    pickle.dump(pipeline, model_file)

In [15]:
# Load the trained model
with open(pickle_model, 'rb') as model_file:
    model = pickle.load(model_file)

# User input for prediction
user_input = {
    'Title': input('Job Title (eg. Data Analyst): '),
    'Location': input('\nLocation (eg. Chester): '),
    'Department': input('\nDepartment (eg. Data): '),
    'Salary Range': input('\nSalary range (eg. 0-10000): '),
    'Company Profile': input('\nCompany profile: '),
    'Description': input('\nJob Description: '),
    'Requirements': input('\nJob Requirements: '),
    'Benefits': input('\nJob Benefits: '),
    'Telecommuting': int(input('\nTelecommuting? (Yes- 1, No- 0): ')),
    'Has Company Logo': int(input('\nCompany Logo? (Yes- 1, No- 0): ')),
    'Has Questions': int(input('\nQuestions? (Yes- 1, No- 0): ')),
    'Employment Type': input('\nEmployment Type (eg. Full-time / Part-time / Fixed Term etc): '),
    'Required Experience': input('\nRequired Experience: '),
    'Required Education': input('\nRequired Education: '),
    'Industry': input('\nIndustry (eg. IT): '),
    'Function': input('\nFunction (eg. Data Analysis): ')
}

# Create a DataFrame from user input
user_data = pd.DataFrame([user_input])

# Make predictions
prediction = model.predict(user_data['Description'])

# Display the prediction
if prediction[0] == 1:
    print("\nThe job listing is predicted to be Fake.\n")
    user_data['Fraudulant'] = 1
else:
    print("\nThe job listing is predicted to be Real.\n")
    user_data['Fraudulant'] = 0

import os

user_data.to_csv(file_path2, mode='a')
print(f"User input and prediction appended to: {file_path2}")

Job Title eg. Data Analyst: Graduate Data Analyst

Location eg. Chester: Manchester, England

Department eg. Data: Data Science

Salary range (eg. 0-10000): 23000

Company profile: 🌟 Grayce…… Our Mission is Your Success 🌟  Recent Graduate in the North West fascinated by the power of data? We are on the lookout for brilliant Graduate Data Analysts RIGHT NOW at Grayce!

Job Description: With your specialism in Data, you will have a tailored learning development journey bespoke to you, meaning you are prepared for whatever the day throws at you whilst learning key skills and gaining industry specific accreditations along the way. Our holistic approach guarantees that you'll emerge as a versatile professional, equipped with skills and experience that truly set you apart.

Job Requirements: 2:1 Undergraduate Degree: Ideally, within a STEM subject where you have handled and manipulated data.   Data Tools: Got a good grip on Excel, Tableau and Python? Maybe you’ve used R, SQL and Power BI in 